In [ ]:
%load_ext autoreload

In [2]:
%autoreload 
import logging
import os
import pathlib
import torch
from ssd.engine.inference import do_evaluation
from ssd.config.defaults import cfg
from ssd.utils.logger import setup_logger
from train import start_train

In [3]:
config_file = "configs/train_rdd2020_server.yaml"

cfg.merge_from_file(config_file)
cfg.freeze()
output_dir = pathlib.Path(cfg.OUTPUT_DIR)
output_dir.mkdir(exist_ok=True, parents=True)

logger = setup_logger("SSD", output_dir)

logger.info("Loaded configuration file {}".format(config_file))
with open(config_file, "r") as cf:
    config_str = "\n" + cf.read()
    logger.info(config_str)
logger.info("Running with config:\n{}".format(cfg))


2021-04-21 18:11:28,596 SSD INFO: Loaded configuration file configs/train_rdd2020_server.yaml
2021-04-21 18:11:28,598 SSD INFO: 
MODEL:
    NUM_CLASSES: 5
    BACKBONE:
        NAME: 'resnet18'
        PRETRAINED: True
        #OUT_CHANNELS: [512, 1024, 2048, 1024, 1024, 512] # resnet152
        OUT_CHANNELS: [128, 256, 512, 1024, 512, 512] # resnet18
        #OUT_CHANNELS: [512, 1024, 512, 512, 256, 256] # model from assignment 4
        INPUT_CHANNELS: 3
    PRIORS:
        # Image size [300, 300]
        #FEATURE_MAPS: [[38, 38], [19, 19], [10, 10], [5, 5], [3, 3], [1, 1]]
        #STRIDES: [[8, 8], [16, 16], [32, 32], [64, 64], [100, 100], [300, 300]]
        #MIN_SIZES: [[30, 30], [60, 60], [111, 111], [162, 162], [213, 213], [264, 264]]
        #MAX_SIZES: [[60, 60], [111, 111], [162, 162], [213, 213], [264, 264], [315, 315]]

        # Image size [600, 600]
        #FEATURE_MAPS: [[75, 75], [38, 38], [19, 19], [10, 10], [5, 5], [3, 3]]
        #MIN_SIZES: [[60, 60], [120, 120], 

In [ ]:
model = start_train(cfg)

Detector initialized. Total Number of params:  38.90M
Backbone number of parameters: 37.65M
SSD Head number of parameters: 1.24M
2021-04-21 20:39:14,208 SSD.trainer INFO: Loading checkpoint from outputs/rdd2020/model_040000.pth
2021-04-21 20:39:14,478 SSD.trainer INFO: Loading optimizer from outputs/rdd2020/model_040000.pth
Dataset loaded. Subset: train, number of images: 19366
2021-04-21 20:39:18,782 SSD.trainer INFO: Start training ...


/home/fredralm/home/TDT4265-StarterCode/Project/SSD/ssd/data/transforms/transforms.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/fredralm/home/TDT4265-StarterCode/Project/SSD/ssd/data/transforms/transforms.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/home/fredralm/home/TDT4265-StarterCode/Project/SSD/ssd/data/transforms/transforms.py:165: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of list

2021-04-21 20:39:22,898 SSD.trainer INFO: iter: 040010, lr: 0.00050, total_loss: 5.144 (5.144), reg_loss: 1.742 (1.742), cls_loss: 3.403 (3.403), time: 0.411 (0.411), eta: 9:07:41, mem: 5865M
2021-04-21 20:39:25,597 SSD.trainer INFO: iter: 040020, lr: 0.00050, total_loss: 4.767 (4.956), reg_loss: 1.719 (1.730), cls_loss: 3.048 (3.225), time: 0.270 (0.340), eta: 7:33:44, mem: 5865M
2021-04-21 20:39:28,603 SSD.trainer INFO: iter: 040030, lr: 0.00050, total_loss: 4.815 (4.909), reg_loss: 1.793 (1.751), cls_loss: 3.023 (3.158), time: 0.301 (0.327), eta: 7:16:00, mem: 5865M
2021-04-21 20:39:31,195 SSD.trainer INFO: iter: 040040, lr: 0.00050, total_loss: 4.690 (4.854), reg_loss: 1.581 (1.709), cls_loss: 3.109 (3.146), time: 0.259 (0.310), eta: 6:53:18, mem: 5865M
2021-04-21 20:39:34,299 SSD.trainer INFO: iter: 040050, lr: 0.00050, total_loss: 4.934 (4.870), reg_loss: 1.727 (1.712), cls_loss: 3.207 (3.158), time: 0.310 (0.310), eta: 6:53:19, mem: 5865M
2021-04-21 20:39:37,046 SSD.trainer INFO

100%|██████████| 482/482 [11:55<00:00,  1.49s/it]


2021-04-21 21:02:34,437 SSD.inference INFO: mAP: 0.0605
D00 - Linear Longitudinal Crack: 0.2314
D10 - Linear Lateral Crack: 0.0002
D20 - Alligator and Other Complex Cracks: 0.0106
D40 - Pothole   : 0.0000

2021-04-21 21:02:36,275 SSD.trainer INFO: iter: 042510, lr: 0.00050, total_loss: 3.880 (4.206), reg_loss: 1.260 (1.439), cls_loss: 2.619 (2.767), time: 72.297 (0.557), eta: 11:59:03, mem: 5865M
2021-04-21 21:02:38,596 SSD.trainer INFO: iter: 042520, lr: 0.00050, total_loss: 4.025 (4.205), reg_loss: 1.343 (1.439), cls_loss: 2.682 (2.766), time: 0.232 (0.555), eta: 11:57:18, mem: 5865M
2021-04-21 21:02:41,202 SSD.trainer INFO: iter: 042530, lr: 0.00050, total_loss: 3.953 (4.204), reg_loss: 1.312 (1.438), cls_loss: 2.640 (2.766), time: 0.261 (0.554), eta: 11:55:42, mem: 5865M
2021-04-21 21:02:43,806 SSD.trainer INFO: iter: 042540, lr: 0.00050, total_loss: 3.963 (4.203), reg_loss: 1.301 (1.438), cls_loss: 2.662 (2.765), time: 0.261 (0.553), eta: 11:54:07, mem: 5865M
2021-04-21 21:02:46,5

...

100%|██████████| 482/482 [08:48<00:00,  1.10s/it]


2021-04-21 22:29:42,271 SSD.inference INFO: mAP: 0.0800
D00 - Linear Longitudinal Crack: 0.2498
D10 - Linear Lateral Crack: 0.0000
D20 - Alligator and Other Complex Cracks: 0.0700
D40 - Pothole   : 0.0000

2021-04-21 22:29:44,005 SSD.trainer INFO: iter: 052510, lr: 0.00050, total_loss: 3.660 (3.910), reg_loss: 1.215 (1.307), cls_loss: 2.445 (2.603), time: 53.651 (0.530), eta: 9:55:42, mem: 5865M
2021-04-21 22:29:46,201 SSD.trainer INFO: iter: 052520, lr: 0.00050, total_loss: 3.899 (3.910), reg_loss: 1.370 (1.307), cls_loss: 2.528 (2.603), time: 0.220 (0.529), eta: 9:55:20, mem: 5865M
2021-04-21 22:29:48,899 SSD.trainer INFO: iter: 052530, lr: 0.00050, total_loss: 3.726 (3.910), reg_loss: 1.256 (1.307), cls_loss: 2.470 (2.603), time: 0.270 (0.529), eta: 9:55:00, mem: 5865M
2021-04-21 22:29:51,293 SSD.trainer INFO: iter: 052540, lr: 0.00050, total_loss: 3.760 (3.910), reg_loss: 1.207 (1.307), cls_loss: 2.553 (2.603), time: 0.239 (0.529), eta: 9:54:40, mem: 5865M
2021-04-21 22:29:54,100 S

100%|██████████| 482/482 [09:36<00:00,  1.20s/it]


2021-04-21 22:50:03,667 SSD.inference INFO: mAP: 0.0725
D00 - Linear Longitudinal Crack: 0.2601
D10 - Linear Lateral Crack: 0.0005
D20 - Alligator and Other Complex Cracks: 0.0295
D40 - Pothole   : 0.0000

2021-04-21 22:50:05,490 SSD.trainer INFO: iter: 055010, lr: 0.00050, total_loss: 3.663 (3.867), reg_loss: 1.128 (1.288), cls_loss: 2.535 (2.580), time: 58.379 (0.523), eta: 9:26:14, mem: 5865M
2021-04-21 22:50:07,689 SSD.trainer INFO: iter: 055020, lr: 0.00050, total_loss: 3.684 (3.867), reg_loss: 1.229 (1.288), cls_loss: 2.455 (2.580), time: 0.220 (0.523), eta: 9:25:56, mem: 5865M
2021-04-21 22:50:10,608 SSD.trainer INFO: iter: 055030, lr: 0.00050, total_loss: 3.599 (3.867), reg_loss: 1.222 (1.288), cls_loss: 2.377 (2.579), time: 0.292 (0.522), eta: 9:25:40, mem: 5865M
2021-04-21 22:50:13,475 SSD.trainer INFO: iter: 055040, lr: 0.00050, total_loss: 3.443 (3.867), reg_loss: 1.120 (1.287), cls_loss: 2.323 (2.579), time: 0.287 (0.522), eta: 9:25:25, mem: 5865M
2021-04-21 22:50:16,229 S

 63%|██████▎   | 306/482 [05:19<03:31,  1.20s/it]

In [5]:
logger.info('Start evaluating...')
torch.cuda.empty_cache()  # speed up evaluating after training finished
do_evaluation(cfg, model)

2021-04-21 18:09:34,587 SSD INFO: Start evaluating...


NameError: name 'model' is not defined